In [ ]:
# -*- coding: utf-8-sig -*-
import pandas as pd 
import numpy as np 
import re

pattern = re.compile('[\u0627-\u064a]')
recipes = pd.read_json('test1000.json',encoding="utf-8-sig")


### extract names from ingredients
ingrdsCorpus = []
for ingrds in recipes["ingrd"]:
    for ingrd in ingrds:
        ingrdTxt = ingrd["n"].replace('/',' ').replace('(',' ').replace(')',' ')
        for comp in ingrdTxt.split():
            if pattern.match(comp) != None:
                ingrdsCorpus.append(comp)
        del ingrdTxt 
        
del recipes

ingrdsCorpus = (set(ingrdsCorpus))



### extract names from prices
prices = pd.read_csv('data.csv',encoding="utf-8-sig")

ItemCorpus = []
for item in prices["n"]:
    item = item.replace('/',' ').replace('(',' ').replace(')',' ')
    for comp in item.split():
        if pattern.match(comp) != None:
            ItemCorpus.append(comp)

    del item  
    
del prices            
ItemCorpus = (set(ItemCorpus))

names = []
for i in ingrdsCorpus:
    if i in ItemCorpus:
        names.append(i) 
names=list(set(names))

print (names)

del ItemCorpus,ingrdsCorpus,names